## class approach

In [1]:
%matplotlib widget
# %matplotlib notebook
import ipywidgets as widgets
from ipywidgets import Layout
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
import random


class app(object):
    def __init__(self):
        self.hys_per_side = 101
        self.beta_values = np.around(np.linspace(0, 1, self.hys_per_side),2) # beta values
        self.alpha_values = np.around(np.linspace(1,0, self.hys_per_side),2) # alpha values
        self.beta_grid, self.alpha_grid = np.meshgrid(self.beta_values, self.alpha_values)
        self.preisach_triangle = np.where(self.alpha_grid>self.beta_grid, 0, np.nan)
        
        
        self.out = widgets.Output()
        self.input_u = widgets.FloatSlider(
                            value=0, 
                            min=0, max=1, step=0.01,
                            description='input $u$',
                            continuous_update=True,
                            layout=Layout(width='50%')
                       )
        self.input_u.observe(self.update_app, 'value')
        self.reset_button = widgets.Button(
                                description='Reset',
                                icon="trash",
                                style={'font_weight': 'bold', 'button_color': 'yellow'}
                            )
        self.reset_button.on_click(self.on_button_clicked)# if you click it will activate the function
        self.on_button_clicked(1) # We force a click to reset all the plots
    
    def on_button_clicked(self, b): # its just the jupiter format for clicking
        plt.clf()
        self.fig = plt.figure(1, figsize=(15,5))
        # self.fig.canvas.layout.width = '1000px'
        gs1 = GridSpec(3, 1, width_ratios=[1],
                             height_ratios=[1, 1, 1],
                             left=0.03, right=0.36,
                             bottom=0.03, top=1.00,
                             wspace=0.6, hspace=0.2)
        gs2 = GridSpec(2, 1, width_ratios=[1],
                             height_ratios=[1, 1],
                             left=0.65, right=0.95,
                             bottom=0.03, top=0.92,
                             wspace=0.6, hspace=0.5)
        self.ax1 = plt.subplot(gs1[0, 0])
        self.ax2 = plt.subplot(gs1[1, 0])
        self.ax3 = plt.subplot(gs1[2, 0])
        self.ax4 = plt.subplot(gs2[0, 0])
        self.ax5 = plt.subplot(gs2[1, 0])
        # x is the same as the input u, but it's a static array, used for plotting
        self.x = np.arange(0, 1, 0.005)
        
        # Setting all 3 hysterons
        self.reset_AB() # randomly draw alpha and beta for the hysterons, then plot
        self.plot_hysteron(self.ax1, self.A1, self.B1, 1)
        self.plot_hysteron(self.ax2, self.A2, self.B2, 2)
        self.plot_hysteron(self.ax3, self.A3, self.B3, 3)        
        
        # Hysteresis plot
        self.ax4.axis([-0.05, 1.2, -0.3, 3.5])
        self.ax4.annotate('Output', xy=(-0.05, -0.2), xycoords=('data'),                   # arrow up
                          xytext=(-0.05, 3.8), textcoords='data',
                          ha='center', va='bottom',
                          arrowprops=dict(arrowstyle='<|-', fc='black'))
        self.ax4.annotate('', xy=(-0.05, -0.2), xycoords=('data'),                        # arrow right
                          xytext=(1.2, -0.2), textcoords='data',
                          ha='left', va='center',
                          arrowprops=dict(arrowstyle='<|-', fc='black'))
        self.ax4.text(1.2, 0.1, "$u$", ha="right")
        self.ax4.spines['top'].set_visible(False)
        self.ax4.spines['right'].set_visible(False)
        self.ax4.spines['bottom'].set_visible(False)
        self.ax4.spines['left'].set_visible(False)
        self.ax4.set_xticks([])
        self.ax4.set_yticks([])
        self.ax4.tick_params(axis=u'both', which=u'both',length=0)

        # Preisach triangle plot
        self.ax5.axis([0.0, 1.2, -0.05, 1.1])
        self.ax5.plot(self.x, self.x, lw=1, dashes=[6, 2], color="black")
        self.ax5.scatter([self.B1, self.B2, self.B3], [self.A1, self.A2, self.A3], color='red', s=15,zorder=3)
        self.ax5.annotate("1", (self.B1 + 0.03, self.A1 - 0.015),color="red",zorder=3, size=10)
        self.ax5.annotate("2", (self.B2 + 0.03, self.A2 - 0.015),color="red",zorder=3, size=10)
        self.ax5.annotate("3", (self.B3 + 0.03, self.A3 - 0.015),color="red",zorder=3, size=10)
        self.ax5.text(0.5,1.1,"Limit Triangle", ha="center")
        self.ax5.annotate('', xy=(0, 0), xycoords=('data'),                        # arrow up
                          xytext=(0, 1.2), textcoords='data',
                          ha='center', va='bottom',
                          arrowprops=dict(arrowstyle='<|-', fc='black'))
        self.ax5.text(0.0, 1.22, r"$\alpha$")                                      # arrow right
        self.ax5.annotate('', xy=(0, 0), xycoords=('data'),
                          xytext=(1.2, 0), textcoords='data',
                          ha='left', va='center',
                          arrowprops=dict(arrowstyle='<|-', fc='black'))
        self.ax5.text(1.2, 0.05, r"$\beta$")
        self.ax5.annotate('', xy=(0, 0), xycoords=('data'),                        # diagonal arrow
                          xytext=(1.1, 1.1), textcoords='data',
                          ha='left', va='center',
                          arrowprops=dict(arrowstyle='<|-', fc='black'))
        self.ax5.text(1.0, 0.93, r"$u$")
        self.ax5.spines['top'].set_visible(False)
        self.ax5.spines['right'].set_visible(False)
        self.ax5.spines['bottom'].set_visible(False)
        self.ax5.spines['left'].set_visible(False)
        self.ax5.set_xticks([])
        self.ax5.set_yticks([])
        self.ax5.tick_params(axis=u'both', which=u'both',length=0)
        self.ax5.fill_between(self.x, self.x, y2=1.0, color="gray", alpha=0.2,zorder=1)
        self.on_state = self.ax5.fill_between([0], [0], 0, color="black",zorder=0)
        
        
        self.input_u.value = 0.0
        self.input_history = [0.0]
        self.output_history = [0.0]
        
        self.reset_markers()
        
    def plot_hysteron(self, ax, A, B, hysteron_number):
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.annotate('$u$', xy=(0, 0), xycoords=('data'),
                      xytext=(1.2, 0), textcoords='data',
                      ha='left', va='center', zorder=-1,
                      arrowprops=dict(arrowstyle='<|-', fc='black'))
        ax.annotate('', xy=(0, 0), xycoords=('data'),
                      xytext=(0, 1.5), textcoords='data',
                      ha='center', va='bottom', zorder=-1,
                      arrowprops=dict(arrowstyle='<|-', fc='black'))
        ax.plot(np.where(self.x < A , self.x,np.nan), np.where(self.x < A , 0,np.nan), zorder=1,color="orange")
        ax.plot(np.where(self.x > A , self.x,np.nan), np.where(self.x > A , 1,np.nan), zorder=1,color="orange")
        ax.plot([A,A], [0,1], zorder=1,color="orange")
        ax.plot(np.where(self.x < B , self.x,np.nan), np.where(self.x < B , 0,np.nan), zorder=1,color="blue")
        ax.plot(np.where(self.x > B , self.x,np.nan), np.where(self.x > B , 1,np.nan), zorder=1,color="blue")
        ax.plot([B,B], [0,1], zorder=1,color="blue")
        ax.plot([A],[0.5],marker="^",color="orange",zorder=0)
        ax.plot([B],[0.5],marker="v",color="blue",zorder=0)
        ax.text(A, -0.25, r"$\alpha$", horizontalalignment="center")
        ax.text(B, -0.25, r"$\beta$", horizontalalignment="center")
        ax.axis([0, 1, -0.2, 1.5])
        ax.text(0.5,1.3,"hysteron {}".format(hysteron_number), ha="center")

    def reset_AB(self):
        """
        Resetting all Alphas and Betas
        """
        A_vec = []
        B_vec = []
        for i in range(3):
            beta  = np.random.randint(1, 9) # beta between 1 and 8
            alpha = np.random.randint(beta+1, 10) # alpha between beta+1 and 9
            A_vec.append(0.1*alpha)
            B_vec.append(0.1*beta)
        self.A1, self.A2, self.A3 = A_vec
        self.B1, self.B2, self.B3 = B_vec

    def reset_markers(self):
        self.H1 = 0.0
        self.H2 = 0.0
        self.H3 = 0.0
        self.marker1, = self.ax1.plot([0], self.H1, 'ro', zorder=2, clip_on=False)
        self.marker2, = self.ax2.plot([0], self.H2, 'ro', zorder=2, clip_on=False)
        self.marker3, = self.ax3.plot([0], self.H3, 'ro', zorder=2, clip_on=False)
        self.marker_hysteresis, = self.ax4.plot([0], self.H3, 'ro', zorder=5, clip_on=False)
        self.hysteresis_curve, = self.ax4.plot(self.input_history, self.output_history, color='green')
    
    def update_hysteron_value(self, u0, u1, alpha, beta, previous_state):
        """
        This is the core of the algorithm
        The value of the hysteron can only change if there is a threshold crossing.
        If the input increased (u1-u0>0), the threshold is alpha, otherwise beta.
        
        Case (a): input increased
        If the input is to the right of alpha, then the histeron is surely on, no matter what was its previous state
        Case (b): input decreased
        If the input is to the left of beta, then the histeron is surely off, no matter what was its previous state
        Is case we are neither to the right of alpha nor to the left of beta, then the hysteron remain as it was.
        """
        if (u1 - u0) > 0:
            threshold = alpha
            if u1 >= threshold:
                return 1.0
        else:
            threshold = beta
            if u1 <= threshold:
                return 0.0
        return previous_state
    def RegularPreisach(self, u):
        alpha = self.alpha_grid
        beta = self.beta_grid
        preisach_triangle = self.preisach_triangle
        # compare new input to previous input value and change hysteron values accordingly
        """This if u.new==0 is in order to cancel the last thin black patch in the left side
         of the priesach trianle and make it a whole gray when u=0"""
        if u.new==0:
                preisach_triangle=np.where(self.alpha_grid>self.beta_grid, 0, np.nan)
                preisach_triangle[-1][0]=1
        elif u.new > u.old: # if input increases  
            preisach_triangle = np.where(u.new>alpha, 1, preisach_triangle)
        elif u.new < u.old: # if input increases
            preisach_triangle = np.where(u.new<beta, 0, preisach_triangle)
            

        # values outside the presiach half-plane are set to nan
        preisach_triangle = np.where(alpha>=beta, preisach_triangle, np.nan)
        return preisach_triangle
    def xy_on(self, triangle):
        m = triangle 
        d = np.diff(m,axis=0)  # Calculate the difference between an element and the above
        x = np.where(d==1)[1]  # Getting all the x-columns coordinate
        y = self.hys_per_side-np.where(d==1)[0]-1  # Getting all the y-raws coordinate
        x = np.append(x, x[-1]+1)  # adding missing point at...
        y = np.append(y, y[-1])    # the end of the array
        step_index = np.where(np.diff(y)!= 0)[0]
        x = np.insert(x, step_index+1, x[step_index+1])
        y = np.insert(y, step_index, y[step_index])
        return [x/(self.hys_per_side-1), y/(self.hys_per_side-1)]
    
    def update_app(self, u):
        self.preisach_triangle = self.RegularPreisach(u)
        
        #update hysterons
        self.H1 = self.update_hysteron_value(u.old, u.new, self.A1, self.B1, self.H1)
        self.H2 = self.update_hysteron_value(u.old, u.new, self.A2, self.B2, self.H2)
        self.H3 = self.update_hysteron_value(u.old, u.new, self.A3, self.B3, self.H3)
        self.marker1.set_data([u.new], [self.H1])
        self.marker2.set_data([u.new], [self.H2])
        self.marker3.set_data([u.new], [self.H3])
        # update hysteresis graph
        self.input_history.append(u.new)
        self.output_history.append(self.H1 + self.H2 + self.H3)
        self.hysteresis_curve.set_data(self.input_history, self.output_history)
        self.marker_hysteresis.set_data([self.input_history[-1]], [self.output_history[-1]])
        self.on_state.remove()
        x_on, y_on = self.xy_on(self.preisach_triangle)
        self.on_state = self.ax5.fill_between(x_on, x_on, y_on, color="black")
my_app = app()
widgets.HBox([my_app.reset_button, my_app.input_u])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …